In [1]:
import os
import sys

PACKAGE_PARENT = '..'
SCRIPT_DIR = os.path.dirname(os.path.realpath(os.path.join(os.getcwd())))
#sys.path.append(os.path.normpath(os.path.join(SCRIPT_DIR, PACKAGE_PARENT)))

sys.path.append(SCRIPT_DIR)
print(SCRIPT_DIR)

/Users/Michi/Dropbox/Local/Physics_projects/GWfish


In [2]:
import copy
import numpy as onp

In [3]:
import GWfish.Globals as glob

In [4]:
#import jax

In [5]:
#XLA_PYTHON_CLIENT_MEM_FRACTION=.2

## COMPLETE EXAMPLE: GW170817

In [4]:
# We use the positions of existing LVC detectors 
from GWfish.Globals import detectors as base_dets

LVKdetectors = copy.deepcopy(base_dets)

In [5]:
# We use the O2 psds
LVKdetectors['L1']['psd_path'] =os.path.join(glob.detPath, 'LVK_O1O2O3', '2017-08-06_DCH_C02_L1_O2_Sensitivity_strain_asd.txt')
LVKdetectors['H1']['psd_path'] =os.path.join(glob.detPath, 'LVK_O1O2O3', '2017-06-10_DCH_C02_H1_O2_Sensitivity_strain_asd.txt')
LVKdetectors['Virgo']['psd_path'] =os.path.join(glob.detPath, 'LVK_O1O2O3', 'Hrec_hoft_V1O2Repro2A_16384Hz.txt')





In [6]:
from GWfish.waveforms import ReducedPN_TaylorF2_BNS
from GWfish.signal import GWSignal
from GWfish.network import DetNet
from fisherTools import check_covariance


In [7]:

myLVKSignals = {}

for d in LVKdetectors.keys():

    myLVKSignals[d] = GWSignal( ReducedPN_TaylorF2_BNS(), 
                psd_path=LVKdetectors[d]['psd_path'],
                detector_shape = LVKdetectors[d]['shape'],
                det_lat= LVKdetectors[d]['lat'],
                det_long=LVKdetectors[d]['long'],
                det_xax=LVKdetectors[d]['xax'], 
                verbose=True,
                useEarthMotion = False,
                fmin=10.,
                IntTablePath=None) 
        #os.path.join('../data/New_ET_Curves_11-16/ASD_ETtest/HFLF_cryo/', 'Integral_Tables', 'NewtInspiralBNS200.h5'))

myLVKNet = DetNet(myLVKSignals)      

Using PSD from file /Users/Michi/Dropbox/Local/Physics_projects/GWfish/psds/LVK_O1O2O3/2017-08-06_DCH_C02_L1_O2_Sensitivity_strain_asd.txt 
Using PSD from file /Users/Michi/Dropbox/Local/Physics_projects/GWfish/psds/LVK_O1O2O3/2017-06-10_DCH_C02_H1_O2_Sensitivity_strain_asd.txt 
Using PSD from file /Users/Michi/Dropbox/Local/Physics_projects/GWfish/psds/LVK_O1O2O3/Hrec_hoft_V1O2Repro2A_16384Hz.txt 


In [58]:
from utils import th_phi_from_ra_dec, deg_min_sec_to_decimal_deg, hr_min_sec_to_decimal_deg

decGW170817 = deg_min_sec_to_decimal_deg(-23, 23, 2)
raGW170817 =  hr_min_sec_to_decimal_deg(13, 9, 47)

thGW170817, phiGW170817 = th_phi_from_ra_dec(raGW170817, decGW170817)

m1GW170817 = 1.46
m2GW170817 = 1.27
mTotGW170817 = m1GW170817+m2GW170817
etaGW170817 = m1GW170817*m2GW170817/mTotGW170817**2


GW170817 = {'Mc': onp.array([1.186*(1+0.01) ,]) , # in detector frame  
            'dL': onp.array([40/1000,]), # in Gpc
            'theta': onp.array([thGW170817,]),  
            'phi': onp.array([phiGW170817,]),
            'iota': onp.array([146*onp.pi/180],),
            'psi': onp.array([0.,]), #onp.random.uniform(0, np.pi*2, 1),
            'tcoal': onp.zeros(1), #onp.random.uniform(0, 1, 1), 
            'eta': onp.array([etaGW170817, ]), #onp.random.uniform(0, 0.25, 1), 
            'Phicoal': onp.zeros(1), #onp.random.uniform(0, np.pi*2, 1)
           }

GW170817 

{'Mc': array([1.19786]),
 'dL': array([0.04]),
 'theta': array([1.96552193]),
 'phi': array([3.44607989]),
 'iota': array([2.54818071]),
 'psi': array([0.]),
 'tcoal': array([0.]),
 'eta': array([0.24878906]),
 'Phicoal': array([0.])}

In [59]:
# Compute the expected SNR (true was 33 )
myLVKNet.SNR(GW170817)

array([12.67631999])

In [60]:
# Compute the total Fisher matrix. Here the derivatives are wrt dL
totF = myLVKNet.FisherMatr(GW170817, res=1000)


Computing Fisher for L1...

Computing Fisher for H1...

Computing Fisher for Virgo...

Computing total Fisher ...
Condition number= [1.5248049e+14] . Ok. 
Done.


In [64]:
# Compute the total Fisher matrix

from fisherTools import log_dL_to_dL_derivative_fish, CheckFisher

# This Fisher is wrt dL, using Fisher wrt dL, converted before inversion
totF_dL = log_dL_to_dL_derivative_fish(totF, {'logdL':1}, GW170817)

In [65]:
totF[1, 1]/0.04**2

array([164780.00365439])

In [66]:
totF_dL[1, 1]

array([164780.00365439])

#### 1 - compute covariance in logdL

In [14]:
# Compute covariance matrix

# This covariance is wrt logdL
Cov, parNums, eps, _ = myLVKNet.CovMatr(GW170817, ParMarg=None, FisherM=totF, 
                                     invMethod='inv', 
                                     return_inv_err=True, 
                                     get_individual=False,
                                     return_dL_derivs=False)


Computing total covariance ...
Condition number= [1.52163305e+14] . Ok. 
Done.
Inversion error: [0.009480738964702367] 


In [15]:
II = check_covariance(totF, Cov)

Inversion errors: [0.009480738964702367]
diagonal-1 = [array([-1.03842467e-07, -1.73268067e-11,  3.37564643e-09,  1.31319666e-09,
        5.98488725e-09, -6.51198206e-10, -2.18278728e-11,  3.08267772e-07,
        4.51109372e-10])]
Max off diagonal: [0.000948609784245491]

mask: where F*S(off-diagonal)>1e-10 (--> problematic if True off diagonal)
[array([[ True, False,  True, False, False, False,  True,  True, False],
       [ True,  True, False,  True,  True, False, False, False,  True],
       [False, False,  True, False, False,  True,  True,  True, False],
       [ True, False, False,  True, False, False, False, False,  True],
       [ True, False, False,  True,  True, False, False, False,  True],
       [ True, False, False,  True, False,  True, False, False,  True],
       [ True, False, False, False, False, False,  True, False, False],
       [False, False, False, False, False, False,  True,  True, False],
       [ True, False, False,  True, False, False,  True, False,  True]])]


#### 2 - compute covariance by inverting fisher in logdL and then switching to dL derivative

In [16]:
# Compute covariance matrix

# This covariance is wrt dL, using Fisher wrt logdL and converting after inversions
Cov_dL, parNums_dL, eps_dL, _ = myLVKNet.CovMatr(GW170817, ParMarg=None, FisherM=totF, 
                                     invMethod='inv', 
                                     return_inv_err=True, 
                                    get_individual=False,
                                     return_dL_derivs=True)


Computing total covariance ...
Condition number= [1.52163305e+14] . Ok. 
Switching from derivatives wrt logdL to derivatives wrt dL...
Done.
Converting Fisher to dL to check inversion error...
Inversion error: [0.009480739686691166] 


In [17]:
II = check_covariance(totF_dL, Cov_dL)

Inversion errors: [0.009480739686691166]
diagonal-1 = [array([-1.03842467e-07, -1.73250303e-11,  3.37564643e-09,  1.31319666e-09,
        5.98488725e-09, -6.51198206e-10, -2.18278728e-11,  3.08267772e-07,
        4.51109372e-10])]
Max off diagonal: [0.000948609784245491]

mask: where F*S(off-diagonal)>1e-10 (--> problematic if True off diagonal)
[array([[ True, False,  True, False, False, False,  True,  True, False],
       [ True,  True, False,  True,  True, False, False, False,  True],
       [False, False,  True, False, False,  True,  True,  True, False],
       [ True, False, False,  True, False, False, False, False,  True],
       [ True, False, False,  True,  True, False, False, False,  True],
       [ True, False, False,  True, False,  True, False, False,  True],
       [ True, False, False, False, False, False,  True, False, False],
       [False, False, False, False, False, False,  True,  True, False],
       [ True, False, False,  True, False, False,  True, False,  True]])]


#### 3 - compute covariance by inverting Fisher in dL

In [18]:
_ = CheckFisher(totF_dL)

Condition number= [1.52095382e+14] . Ok. 


In [19]:
# Compute covariance matrix

Cov_from_dL_fish, _, _, _ = myLVKNet.CovMatr(GW170817, ParMarg=None, FisherM=totF_dL, 
                                     invMethod='inv', 
                                     return_inv_err=True, 
                                     get_individual=False,
                                     return_dL_derivs=False)


Computing total covariance ...
Condition number= [1.52095382e+14] . Ok. 
Done.
Inversion error: [0.0029610963701489144] 


In [20]:
(Cov_from_dL_fish - Cov_dL).max()

1.91281505301788e-07

In [33]:
_ = check_covariance(totF_dL, Cov_from_dL_fish)

Inversion errors: [0.0029610963701489144]
diagonal-1 = [array([ 3.52971256e-07,  2.36788367e-12,  1.70803105e-09,  1.84400051e-10,
        1.87375804e-09,  1.60071068e-10,  1.65891834e-09, -1.02911144e-07,
       -1.65891834e-09])]
Max off diagonal: [0.00029608234763145447]

mask: where F*S(off-diagonal)>1e-10 (--> problematic if True off diagonal)
[array([[ True, False, False,  True, False, False, False, False, False],
       [ True,  True, False,  True, False, False, False, False,  True],
       [False, False,  True, False, False,  True,  True,  True, False],
       [ True, False, False,  True, False, False, False, False, False],
       [ True, False, False,  True,  True, False, False, False,  True],
       [False, False,  True, False, False,  True,  True,  True, False],
       [False, False, False, False, False, False,  True,  True, False],
       [ True, False, False, False, False, False, False,  True, False],
       [False, False,  True, False, False,  True,  True,  True,  True]])

#### Perturb Fisher. 

From this check it seems that in order to have a percent level accuracy on the covariance, we need $10^{-4}$ accuracy on the Fisher

In [32]:
# Perturb fisher and check that covariance doesn't change much
from fisherTools import perturb_Fisher 

perturb_Fisher(totF, myLVKNet, GW170817, eps=1e-4)


Computing total covariance ...
Condition number= [1.52163305e+14] . Ok. 
Switching from derivatives wrt logdL to derivatives wrt dL...
Done.

Computing total covariance ...
Condition number= [1.50322625e+14] . Ok. 
Switching from derivatives wrt logdL to derivatives wrt dL...
Done.
Errors: [0.019261441901277987]


#### Forecasted errors

In [42]:
# Forecast error on dL in Mpc
# Observed value is 40 +7 -15

onp.sqrt(Cov[1, 1])*(0.04)*1000

array([37.25969785])

In [35]:
onp.sqrt(Cov_dL[1, 1])*1000

array([37.25969785])

In [45]:
# Forecasted 1 sigma localization area
# Observed is 16 deg squared


from fisherTools import compute_localization_region

compute_localization_region(Cov_dL, parNums_dL, GW170817["theta"], units='SqDeg')

array([48.28499086])

## Single ET

### Configure the interforometer's properties

In [11]:
# We use the positions of existing LVC detectors 
from GWfish.Globals import detectors as base_dets

In [12]:
# Put a single ET at the Virgo site 
ETdet = {'ET': copy.deepcopy(base_dets).pop('Virgo') }

In [13]:
ETdet['ET']['shape'] = 'T'
ETdet['ET']['psd_path'] = os.path.join(glob.detPath, 'pycbc_public', 'et_d.txt')

In [14]:
ETdet

{'ET': {'lat': 43.631,
  'long': 10.504,
  'xax': 71.0,
  'shape': 'T',
  'psd_path': '/Users/Michi/Dropbox/Local/Physics_projects/GWfish/psds/pycbc_public/et_d.txt'}}

### Build the DetNet object

In [15]:
from GWfish.waveforms import ReducedPN_TaylorF2_BNS
from GWfish.signal import GWSignal
from GWfish.network import DetNet

In [16]:
mySignalsET = {}

for d in ETdet.keys():

    mySignalsET[d] = GWSignal( ReducedPN_TaylorF2_BNS(), 
                psd_path= ETdet[d]['psd_path'],
                detector_shape = ETdet[d]['shape'],
                det_lat= ETdet[d]['lat'],
                det_long=ETdet[d]['long'],
                det_xax=ETdet[d]['xax'], 
                verbose=True,
                useEarthMotion = True,
                fmin=2.,
                IntTablePath=None) 
        #os.path.join('../data/New_ET_Curves_11-16/ASD_ETtest/HFLF_cryo/', 'Integral_Tables', 'NewtInspiralBNS200.h5'))

        

myNet = DetNet(mySignalsET) 

Using PSD from file /Users/Michi/Dropbox/Local/Physics_projects/GWfish/psds/pycbc_public/et_d.txt 


In [23]:
from GWfish.tests.test_inversion import sample1d, z_from_dLGW, Om0GLOB
H0fid = 70
Omfid = Om0GLOB
Xi0fid = 1.
nFid = 0.

In [27]:
nevents=500


dLs = sample1d(nevents, lambda x: x**2, 1e-05, 6.6, )
    
zs = z_from_dLGW(dLs*1000, H0fid, Xi0fid, nFid)
    
Mcs = onp.random.uniform(1., 3., nevents)
    
events_rand = {'Mc': Mcs*(1.+zs), 
              'dL': dLs, 
              'theta':onp.random.uniform(-onp.pi/2, onp.pi/2, nevents), 
              'phi':onp.random.uniform(0, onp.pi*2, nevents), 
             'iota':onp.random.uniform(0, onp.pi*2, nevents), 
              'psi':onp.random.uniform(0, onp.pi*2, nevents), 
              'tcoal':onp.random.uniform(0, 1, nevents), 
              'eta': onp.random.uniform(0.24, 0.25, nevents), 
              'Phicoal': onp.random.uniform(0, onp.pi*2, nevents)
             }

In [28]:
%%time
snrs = myNet.SNR(events_rand)

CPU times: user 3.83 s, sys: 274 ms, total: 4.1 s
Wall time: 3.57 s


In [ ]:
%%time
totF = myNet.FisherMatr(events_rand, res=1000)


Computing Fisher for ET...
